# Trexquant Interview Project (The Hangman Game)

* Copyright Trexquant Investment LP. All Rights Reserved. 
* Redistribution of this question without written consent from Trexquant is prohibited

## Instruction:
For this coding test, your mission is to write an algorithm that plays the game of Hangman through our API server. 

When a user plays Hangman, the server first selects a secret word at random from a list. The server then returns a row of underscores (space separated)—one for each letter in the secret word—and asks the user to guess a letter. If the user guesses a letter that is in the word, the word is redisplayed with all instances of that letter shown in the correct positions, along with any letters correctly guessed on previous turns. If the letter does not appear in the word, the user is charged with an incorrect guess. The user keeps guessing letters until either (1) the user has correctly guessed all the letters in the word
or (2) the user has made six incorrect guesses.

You are required to write a "guess" function that takes current word (with underscores) as input and returns a guess letter. You will use the API codes below to play 1,000 Hangman games. You have the opportunity to practice before you want to start recording your game results.

Your algorithm is permitted to use a training set of approximately 250,000 dictionary words. Your algorithm will be tested on an entirely disjoint set of 250,000 dictionary words. Please note that this means the words that you will ultimately be tested on do NOT appear in the dictionary that you are given. You are not permitted to use any dictionary other than the training dictionary we provided. This requirement will be strictly enforced by code review.

You are provided with a basic, working algorithm. This algorithm will match the provided masked string (e.g. a _ _ l e) to all possible words in the dictionary, tabulate the frequency of letters appearing in these possible words, and then guess the letter with the highest frequency of appearence that has not already been guessed. If there are no remaining words that match then it will default back to the character frequency distribution of the entire dictionary.

This benchmark strategy is successful approximately 18% of the time. Your task is to design an algorithm that significantly outperforms this benchmark.

In [1]:
import json
import requests
import random
import string
import secrets
import time
import re
import collections

try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode

from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [2]:
class HangmanAPI(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        self.hangman_url = self.determine_hangman_url()
        self.access_token = access_token
        self.session = session or requests.Session()
        self.timeout = timeout
        self.guessed_letters = []
        
        full_dictionary_location = "words_250000_train.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)        
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()
        
        self.current_dictionary = []
        
    @staticmethod
    def determine_hangman_url():
        links = ['https://trexsim.com', 'https://sg.trexsim.com']

        data = {link: 0 for link in links}

        for link in links:

            requests.get(link)

            for i in range(10):
                s = time.time()
                requests.get(link)
                data[link] = time.time() - s

        link = sorted(data.items(), key=lambda x: x[1])[0][0]
        link += '/trexsim/hangman'
        return link


    ## i passed another parameter, tries_remains
    def guess(self, word,tries_remains): # word input example: "_ p p _ e "
        full_dictionary = self.full_dictionary
        word_len = len(word)
        sorted_word_on_len = [word for word in full_dictionary if word_len == len(word)]

        # to word count for each word, to get a Term frequency of word so that i can guess better character
        list_of_dict = []
        for word in sorted_word_on_len:
            char_dict = {}
            for char in word:
                char_dict[char] = word.count(char)
                list_of_dict.append(char_dict)
        # make prototype of each char, to better logic implementation 
        demo_dict = {'a':0,'b':0,'c':0,'d':0,'e':0,'f':0,'g':0,'h':0,'i':0,'j':0,'k':0,'l':0,'m':0,'n':0,
                     'o':0,'p':0,'q':0,'r':0,'s':0,'t':0,'u':0,'v':0,'w':0,'x':0,'y':0,'z':0}
        for count_dict in list_of_dict:
            for key,value in count_dict.items():
                temp = demo_dict[key]
                demo_dict[key] = temp+value

        # to sort the dictionary based on the value, and value containing term frequency of character
        sorted_dict = dict(sorted(demo_dict.items(), key=lambda x: x[1],reverse=True))

        ## to get sorted character from the dictionary
        sorted_list_of_char = list(sorted_dict.keys())

        first = []
        second=[]
        third=[]
        fourth = []
        fifth = []

        # now again starting filtering according to the status of game
        for char in sorted_list_of_char:
            if (char not in self.guessed_letters) and (tries_remains == 6):
                if len(first) == 0:
                    first.append(char)
                    return char
                else:
                    for charr in first:
                        first_attempt_filter_word_list = []
                        for word in sorted_word_on_len:
                            if char in word:
                                first_attempt_filter_word_list.append(word)

                        list_of_dict = []
                        for word in first_attempt_filter_word_list:
                            char_dict = {}
                            for char in word:
                                char_dict[char] = word.count(char)
                                list_of_dict.append(char_dict)

                        demo_dict = {'a':0,'b':0,'c':0,'d':0,'e':0,'f':0,'g':0,'h':0,'i':0,'j':0,'k':0,'l':0,'m':0,'n':0,
                                    'o':0,'p':0,'q':0,'r':0,'s':0,'t':0,'u':0,'v':0,'w':0,'x':0,'y':0,'z':0}
                        for count_dict in list_of_dict:
                            for key,value in count_dict.items():
                                temp = demo_dict[key]
                                demo_dict[key] = temp+value

                        # to sort the dict based on the value
                        sorted_dict = dict(sorted(demo_dict.items(), key=lambda x: x[1],reverse=True))
                        advan_filter_char_list = list(sorted_dict.keys())
                        for char in self.guessed_letters:
                            advan_filter_char_list.remove(char)

                    for char in advan_filter_char_list:
                        if char not in self.guessed_letters:
                            first.append(char)
                            return char

            elif tries_remains == 5:
                # analyses correct and wrong guess char in first attempt
                wrong_guessed_in_first_attempt = []
                correct_guessed = []
                if (len(first)==1) and (len(first)>1):
                    wrong_guessed_in_first_attempt.append(first[0])
                    wrong_guessed_in_first_attempt.append(first[-1])
                    for i in range(1,len(first)-1):
                        correct_guessed.append(first[i])
                    
                ## filter again the dictionary based on the correct and wrong guessed char

                # filter word based on wrong char guessed
                filter_list_worng_base = []
                for char in wrong_guessed_in_first_attempt:
                    for word in sorted_word_on_len:
                        if char not in word:
                            filter_list_worng_base.append(word)

                # again filter word based on correct guessed char
                filter_list_correct_base = []
                for char in correct_guessed:
                    for word in sorted_word_on_len:
                        if char  in word:
                            filter_list_correct_base.append(word)

                ## combine both filter words
                word_list_for_fifth_attempt = filter_list_worng_base + filter_list_correct_base

                list_of_dict = []
                for word in word_list_for_fifth_attempt:
                    char_dict = {}
                    for char in word:
                        char_dict[char] = word.count(char)
                        list_of_dict.append(char_dict)

                demo_dict = {'a':0,'b':0,'c':0,'d':0,'e':0,'f':0,'g':0,'h':0,'i':0,'j':0,'k':0,'l':0,'m':0,'n':0,
                            'o':0,'p':0,'q':0,'r':0,'s':0,'t':0,'u':0,'v':0,'w':0,'x':0,'y':0,'z':0}
                for count_dict in list_of_dict:
                    for key,value in count_dict.items():
                        temp = demo_dict[key]
                        demo_dict[key] = temp+value

                # to sort the dict based on the value
                sorted_dict = dict(sorted(demo_dict.items(), key=lambda x: x[1],reverse=True))
                advan_filter_char_list = list(sorted_dict.keys())
                for char in self.guessed_letters:
                    advan_filter_char_list.remove(char)

                for char in advan_filter_char_list:
                    if char not in self.guessed_letters:
                        second.append(char)
                        return char
                          
            elif tries_remains == 4:
                # analyses correct and wrong guess char in first attempt
                wrong_guessed_in_second_attempt = []
                correct_guessed_second_attempt = []
                if (len(second)==1) and (len(second)>1):
                    wrong_guessed_in_second_attempt.append(second[0])
                    wrong_guessed_in_second_attempt.append(second[-1])
                    for i in range(1,len(second)-1):
                        correct_guessed_second_attempt.append(second[i])

                    # concatenate wrong and correct
                    wrng_word_list_for_fourth_attempt = wrong_guessed_in_first_attempt + wrong_guessed_in_second_attempt
                    wrng_word_list_for_fourth_attempt = list(set(wrng_word_list_for_fourth_attempt))

                    corect_word_list_fourth_attempt = correct_guessed+correct_guessed_second_attempt 
                    corect_word_list_fourth_attempt = list(set(corect_word_list_fourth_attempt))

                    # filter word based on wrong char guessed
                    filter_list_worng_base = []
                    for char in wrng_word_list_for_fourth_attempt:
                        for word in sorted_word_on_len:
                            if char not in word:
                                filter_list_worng_base.append(word)

                    # again filter word based on correct guessed char
                    filter_list_correct_base = []
                    for char in corect_word_list_fourth_attempt:
                        for word in sorted_word_on_len:
                            if char  in word:
                                filter_list_correct_base.append(word)

                    ## combine both filter words
                    word_list_for_fifth_attempt = filter_list_worng_base + filter_list_correct_base

                    list_of_dict = []
                    for word in word_list_for_fifth_attempt:
                        char_dict = {}
                        for char in word:
                            char_dict[char] = word.count(char)
                            list_of_dict.append(char_dict)

                demo_dict = {'a':0,'b':0,'c':0,'d':0,'e':0,'f':0,'g':0,'h':0,'i':0,'j':0,'k':0,'l':0,'m':0,'n':0,
                            'o':0,'p':0,'q':0,'r':0,'s':0,'t':0,'u':0,'v':0,'w':0,'x':0,'y':0,'z':0}
                for count_dict in list_of_dict:
                    for key,value in count_dict.items():
                        temp = demo_dict[key]
                        demo_dict[key] = temp+value

                # # to sort the dict based on the value
                sorted_dict = dict(sorted(demo_dict.items(), key=lambda x: x[1],reverse=True))
                advan_filter_char_list = list(sorted_dict.keys())
                for char in self.guessed_letters:
                    advan_filter_char_list.remove(char)

                for char in advan_filter_char_list:
                    if char not in self.guessed_letters:
                        third.append(char)
                        return char

            elif tries_remains == 3:
                # analyses correct and wrong guess char in first attempt
                wrong_guessed_in_third_attempt = []
                correct_guessed_third_attempt = []
                if (len(third)==1) and (len(third)>1):
                    wrong_guessed_in_third_attempt.append(third[0])
                    wrong_guessed_in_third_attempt.append(third[-1])
                    for i in range(1,len(third)-1):
                        correct_guessed_third_attempt.append(third[i])

                    # concatenate wrong and correct
                    wrng_word_list_for_third_attemp =  wrong_guessed_in_third_attempt + wrong_guessed_in_first_attempt + wrong_guessed_in_second_attempt
                    wrng_word_list_for_third_attemp = list(set(wrng_word_list_for_third_attemp))

                    corect_word_list_third_attemp = correct_guessed_third_attempt + correct_guessed + correct_guessed_second_attempt
                    corect_word_list_third_attemp = list(set(corect_word_list_third_attemp))


                    # filter word based on wrong char guessed
                    filter_list_worng_base = []
                    for char in wrng_word_list_for_third_attemp:
                        for word in sorted_word_on_len:
                            if char not in word:
                                filter_list_worng_base.append(word)

                    # again filter word based on correct guessed char
                    filter_list_correct_base = []
                    for char in corect_word_list_third_attemp:
                        for word in sorted_word_on_len:
                            if char  in word:
                                filter_list_correct_base.append(word)

                    ## combine both filter words
                    word_list_for_third_attemp = filter_list_worng_base + filter_list_correct_base

                    list_of_dict = []
                    for word in word_list_for_third_attemp:
                        char_dict = {}
                        for char in word:
                            char_dict[char] = word.count(char)
                            list_of_dict.append(char_dict)

                demo_dict = {'a':0,'b':0,'c':0,'d':0,'e':0,'f':0,'g':0,'h':0,'i':0,'j':0,'k':0,'l':0,'m':0,'n':0,
                            'o':0,'p':0,'q':0,'r':0,'s':0,'t':0,'u':0,'v':0,'w':0,'x':0,'y':0,'z':0}
                for count_dict in list_of_dict:
                    for key,value in count_dict.items():
                        temp = demo_dict[key]
                        demo_dict[key] = temp+value

                # # to sort the dict based on the value
                sorted_dict = dict(sorted(demo_dict.items(), key=lambda x: x[1],reverse=True))
                advan_filter_char_list = list(sorted_dict.keys())
                for char in self.guessed_letters:
                    advan_filter_char_list.remove(char)

                for char in advan_filter_char_list:
                    if char not in self.guessed_letters:
                        fourth.append(char)
                        return char
            elif tries_remains == 2:
                #analyses correct and wrong guess char in first attempt        <<<<<<<<< fourth se lege
                wrong_guessed_in_two_attempt = []
                correct_guessed_two_attempt = []
                if (len(fourth)==1) and (len(fourth)>1):
                    wrong_guessed_in_two_attempt.append(fourth[0])
                    wrong_guessed_in_two_attempt.append(fourth[-1])
                    for i in range(1,len(fourth)-1):
                        correct_guessed_two_attempt.append(fourth[i])

                    # concatenate wrong and correct
                    wrng_word_list_for_two_attempt = wrong_guessed_in_two_attempt + wrong_guessed_in_third_attempt + wrong_guessed_in_first_attempt + wrong_guessed_in_second_attempt
                    wrng_word_list_for_two_attempt = list(set(wrng_word_list_for_two_attempt))

                    corect_word_list_two_attempt = correct_guessed_two_attempt + correct_guessed_third_attempt + correct_guessed + correct_guessed_second_attempt
                    corect_word_list_two_attempt = list(set(corect_word_list_two_attempt))


                    # filter word based on wrong char guessed
                    filter_list_worng_base = []
                    for char in wrng_word_list_for_two_attempt:
                        for word in sorted_word_on_len:
                            if char not in word:
                                filter_list_worng_base.append(word)

                    # again filter word based on correct guessed char
                    filter_list_correct_base = []
                    for char in corect_word_list_two_attempt:
                        for word in sorted_word_on_len:
                            if char  in word:
                                filter_list_correct_base.append(word)

                    ## combine both filter words
                    word_list_for_two_attempt = filter_list_worng_base + filter_list_correct_base

                    list_of_dict = []
                    for word in word_list_for_two_attempt:
                        char_dict = {}
                        for char in word:
                            char_dict[char] = word.count(char)
                            list_of_dict.append(char_dict)

                demo_dict = {'a':0,'b':0,'c':0,'d':0,'e':0,'f':0,'g':0,'h':0,'i':0,'j':0,'k':0,'l':0,'m':0,'n':0,
                            'o':0,'p':0,'q':0,'r':0,'s':0,'t':0,'u':0,'v':0,'w':0,'x':0,'y':0,'z':0}
                for count_dict in list_of_dict:
                    for key,value in count_dict.items():
                        temp = demo_dict[key]
                        demo_dict[key] = temp+value

                # # to sort the dict based on the value
                sorted_dict = dict(sorted(demo_dict.items(), key=lambda x: x[1],reverse=True))
                advan_filter_char_list = list(sorted_dict.keys())
                for char in self.guessed_letters:
                    advan_filter_char_list.remove(char)

                for char in advan_filter_char_list:
                    if char not in self.guessed_letters:
                        fifth.append(char)
                        return char
            elif tries_remains == 1:
                # analyses correct and wrong guess char in first attempt
                wrong_guessed_in_one_attempt = []
                correct_guessed_one_attempt = []
                if (len(fifth)==1) and (len(fifth)>1):
                    wrong_guessed_in_one_attempt.append(fifth[0])
                    wrong_guessed_in_one_attempt.append(fifth[-1])
                    for i in range(1,len(fifth)-1):
                        correct_guessed_one_attempt.append(fifth[i])

                    # concatenate wrong guessed char
                    wrng_word_list_for_one_attemp = wrong_guessed_in_one_attempt + wrong_guessed_in_two_attempt + wrong_guessed_in_third_attempt + wrong_guessed_in_first_attempt + wrong_guessed_in_second_attempt
                    wrng_word_list_for_one_attemp = list(set(wrng_word_list_for_one_attemp))
                    # concatenate correct guess char
                    corect_word_list_one_attemp = correct_guessed_one_attempt + correct_guessed_two_attempt + correct_guessed_third_attempt + correct_guessed + correct_guessed_second_attempt
                    corect_word_list_one_attemp = list(set(corect_word_list_one_attemp))


                    # filter word based on wrong char guessed
                    filter_list_worng_base = []
                    for char in wrng_word_list_for_one_attemp:
                        for word in sorted_word_on_len:
                            if char not in word:
                                filter_list_worng_base.append(word)

                    # again filter word based on correct guessed char
                    filter_list_correct_base = []
                    for char in corect_word_list_one_attemp:
                        for word in sorted_word_on_len:
                            if char  in word:
                                filter_list_correct_base.append(word)

                    ## combine both filter words
                    word_list_for_one_attemp = filter_list_worng_base + filter_list_correct_base

                    list_of_dict = []
                    for word in word_list_for_one_attemp:
                        char_dict = {}
                        for char in word:
                            char_dict[char] = word.count(char)
                            list_of_dict.append(char_dict)

                demo_dict = {'a':0,'b':0,'c':0,'d':0,'e':0,'f':0,'g':0,'h':0,'i':0,'j':0,'k':0,'l':0,'m':0,'n':0,
                            'o':0,'p':0,'q':0,'r':0,'s':0,'t':0,'u':0,'v':0,'w':0,'x':0,'y':0,'z':0}
                for count_dict in list_of_dict:
                    for key,value in count_dict.items():
                        temp = demo_dict[key]
                        demo_dict[key] = temp+value

                # # to sort the dict based on the value
                sorted_dict = dict(sorted(demo_dict.items(), key=lambda x: x[1],reverse=True))
                advan_filter_char_list = list(sorted_dict.keys())
                for char in self.guessed_letters:
                    advan_filter_char_list.remove(char)

                for char in advan_filter_char_list:
                    if char not in self.guessed_letters:
                        return char
        

    ##########################################################
    # You'll likely not need to modify any of the code below #
    ##########################################################
    
    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary
                
    def start_game(self, practice=True, verbose=True):
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary   # entire list of word
                         
        response = self.request("/new_game", {"practice":practice})
        if response.get('status')=="approved":
            game_id = response.get('game_id')
            word = response.get('word')
            tries_remains = response.get('tries_remains')

            if verbose:
                print("Successfully start a new game! Game ID: {0}. # of tries remaining: {1}. Word: {2}.".format(game_id, tries_remains, word))
            while tries_remains>0:
                # get guessed letter from user code

                # i passed another parameter in the guess function, which is tries_remains
                guess_letter = self.guess(word,tries_remains)
                    
                # append guessed letter to guessed letters field in hangman object
                self.guessed_letters.append(guess_letter)
                if verbose:
                    print("Guessing letter: {0}".format(guess_letter))
                    
                try:    
                    res = self.request("/guess_letter", {"request":"guess_letter", "game_id":game_id, "letter":guess_letter})
                except HangmanAPIError:
                    print('HangmanAPIError exception caught on request.')
                    continue
                except Exception as e:
                    print('Other exception caught on request.')
                    raise e
               
                if verbose:
                    print("Sever response: {0}".format(res))
                status = res.get('status')
                tries_remains = res.get('tries_remains')
                if status=="success":
                    if verbose:
                        print("Successfully finished game: {0}".format(game_id))
                    return True
                elif status=="failed":
                    reason = res.get('reason', '# of tries exceeded!')
                    if verbose:
                        print("Failed game: {0}. Because of: {1}".format(game_id, reason))
                    return False
                elif status=="ongoing":
                    word = res.get('word')
        else:
            if verbose:
                print("Failed to start a new game")
        return status=="success"
        
    def my_status(self):
        return self.request("/my_status", {})
    
    def request(
            self, path, args=None, post_args=None, method=None):
        if args is None:
            args = dict()
        if post_args is not None:
            method = "POST"

        # Add `access_token` to post_args or args if it has not already been
        # included.
        if self.access_token:
            # If post_args exists, we assume that args either does not exists
            # or it does not need `access_token`.
            if post_args and "access_token" not in post_args:
                post_args["access_token"] = self.access_token
            elif "access_token" not in args:
                args["access_token"] = self.access_token

        time.sleep(0.2)

        num_retry, time_sleep = 50, 2
        for it in range(num_retry):
            try:
                response = self.session.request(
                    method or "GET",
                    self.hangman_url + path,
                    timeout=self.timeout,
                    params=args,
                    data=post_args,
                    verify=False
                )
                break
            except requests.HTTPError as e:
                response = json.loads(e.read())
                raise HangmanAPIError(response)
            except requests.exceptions.SSLError as e:
                if it + 1 == num_retry:
                    raise
                time.sleep(time_sleep)

        headers = response.headers
        if 'json' in headers['content-type']:
            result = response.json()
        elif "access_token" in parse_qs(response.text):
            query_str = parse_qs(response.text)
            if "access_token" in query_str:
                result = {"access_token": query_str["access_token"][0]}
                if "expires" in query_str:
                    result["expires"] = query_str["expires"][0]
            else:
                raise HangmanAPIError(response.json())
        else:
            raise HangmanAPIError('Maintype was not text, or querystring')

        if result and isinstance(result, dict) and result.get("error"):
            raise HangmanAPIError(result)
        return result
    
class HangmanAPIError(Exception):
    def __init__(self, result):
        self.result = result
        self.code = None
        try:
            self.type = result["error_code"]
        except (KeyError, TypeError):
            self.type = ""

        try:
            self.message = result["error_description"]
        except (KeyError, TypeError):
            try:
                self.message = result["error"]["message"]
                self.code = result["error"].get("code")
                if not self.type:
                    self.type = result["error"].get("type", "")
            except (KeyError, TypeError):
                try:
                    self.message = result["error_msg"]
                except (KeyError, TypeError):
                    self.message = result

        Exception.__init__(self, self.message)

# API Usage Examples

## To start a new game:
1. Make sure you have implemented your own "guess" method.
2. Use the access_token that we sent you to create your HangmanAPI object. 
3. Start a game by calling "start_game" method.
4. If you wish to test your function without being recorded, set "practice" parameter to 1.
5. Note: You have a rate limit of 20 new games per minute. DO NOT start more than 20 new games within one minute.

In [3]:
token_key = '98a60afeaa4a143b2b4eba935987d6'

api = HangmanAPI(access_token=token_key, timeout=2000)


## Playing practice games:
You can use the command below to play up to 100,000 practice games.

In [4]:

api.start_game(practice=1,verbose=True)
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
practice_success_rate = total_practice_successes / total_practice_runs
print('run %d practice games out of an allotted 100,000. practice success rate so far = %.3f' % (total_practice_runs, practice_success_rate))


Successfully start a new game! Game ID: b7b72f3342ba. # of tries remaining: 6. Word: _ _ _ _ _ _ .
Guessing letter: e
Sever response: {'game_id': 'b7b72f3342ba', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ _ e _ '}
Guessing letter: i
Sever response: {'game_id': 'b7b72f3342ba', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ _ e _ '}
Guessing letter: a
Sever response: {'game_id': 'b7b72f3342ba', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ _ _ _ e _ '}
Guessing letter: n
Sever response: {'game_id': 'b7b72f3342ba', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ _ n n e _ '}
Guessing letter: o
Sever response: {'game_id': 'b7b72f3342ba', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ o n n e _ '}
Guessing letter: r
Sever response: {'game_id': 'b7b72f3342ba', 'status': 'ongoing', 'tries_remains': 3, 'word': '_ o n n e _ '}
Guessing letter: t
Sever response: {'game_id': 'b7b72f3342ba', 'status': 'ongoing', 'tries_remains': 3, 'word': 't o n n e _ '}
Guess

## Playing recorded games:
Please finalize your code prior to running the cell below. Once this code executes once successfully your submission will be finalized. Our system will not allow you to rerun any additional games.

Please note that it is expected that after you successfully run this block of code that subsequent runs will result in the error message "Your account has been deactivated".

Once you've run this section of the code your submission is complete. Please send us your source code via email.

In [5]:

for i in range(1000):
    print('Playing ', i, ' th game')
    # Uncomment the following line to execute your final runs. Do not do this until you are satisfied with your submission
    #api.start_game(practice=0,verbose=False)
    
    # DO NOT REMOVE as otherwise the server may lock you out for too high frequency of requests
    time.sleep(0.5)

Playing  0  th game
Playing  1  th game
Playing  2  th game
Playing  3  th game
Playing  4  th game
Playing  5  th game
Playing  6  th game
Playing  7  th game
Playing  8  th game
Playing  9  th game
Playing  10  th game
Playing  11  th game
Playing  12  th game
Playing  13  th game
Playing  14  th game
Playing  15  th game
Playing  16  th game
Playing  17  th game
Playing  18  th game
Playing  19  th game
Playing  20  th game
Playing  21  th game
Playing  22  th game
Playing  23  th game
Playing  24  th game
Playing  25  th game
Playing  26  th game
Playing  27  th game
Playing  28  th game
Playing  29  th game
Playing  30  th game
Playing  31  th game
Playing  32  th game
Playing  33  th game
Playing  34  th game
Playing  35  th game
Playing  36  th game
Playing  37  th game
Playing  38  th game
Playing  39  th game
Playing  40  th game
Playing  41  th game
Playing  42  th game
Playing  43  th game
Playing  44  th game
Playing  45  th game
Playing  46  th game
Playing  47  th game
Pl

## To check your game statistics
1. Simply use "my_status" method.
2. Returns your total number of games, and number of wins.

In [7]:
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
success_rate = total_recorded_successes/total_recorded_runs
print('overall success rate = %.3f' % success_rate)

overall success rate = 0.000
